In [20]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Flatten
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [3]:
Corpus = pd.read_csv(r"D:\\NLP project\\dataset\\final\\newcleandata2.csv",encoding='latin-1')
Corpus['title'].dropna(inplace=True)

In [4]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['title'],Corpus['label'],test_size=0.2)

In [5]:

print(len(Train_X))
print(len(Train_Y))
print(len(Test_X))
print(len(Test_Y))

1934957
1934957
483740
483740


In [6]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))


In [13]:
training_labels_final = np.array(Train_Y)
testing_labels_final = np.array(Test_Y)

In [7]:
vocab_size = 700000
max_length = 200
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(Train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(Train_X)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(Test_X)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [25]:

embeddings_index = {};
with open('D:\\NLP project\\dataset\\final\\glove.6B.100d.txt',encoding="utf-8") as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1,100));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [26]:
model = Sequential()
model.add(Embedding(vocab_size+1, 100, input_length=max_length, weights=[embeddings_matrix], trainable=False))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 100)          70000100  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200, 128)          84480     
_________________________________________________________________
flatten_2 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25601     
Total params: 70,110,181
Trainable params: 110,081
Non-trainable params: 70,000,100
_________________________________________________________________
None


In [28]:
num_epochs =2
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 1934957 samples, validate on 483740 samples
Epoch 1/2
1934957/1934957 [==============================] - 8944s 5ms/step - loss: 0.0324 - acc: 0.9893 - val_loss: 0.0246 - val_acc: 0.9919
Epoch 2/2
1934957/1934957 [==============================] - 9619s 5ms/step - loss: 0.0226 - acc: 0.9927 - val_loss: 0.0230 - val_acc: 0.9927


In [37]:
#saving
import pickle
with open('D:\\NLP project\\LSTMFinal model-Copy1tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


In [36]:
model.save('D:\\NLP project\\models\\LSTMFinal model-Copy1')


In [35]:
padding_type='post'
sentence = ["i'm going to reach my goal"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
x=model.predict(padded)
print(x)
if x>0.5:
    print("inapprpriate context")
else:
    print("appropriate context")

[[0.01262252]]
appropriate context


In [10]:
import pickle 
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 120
trunc_type='post'
f = open('D:/NLP project/models/tokenizer.pickle', 'rb')
tokenizer = pickle.load(f)
model =tf.keras.models.load_model('D:/NLP project/models/LSTMFinal Model')
padding_type='post'
sentence = ["when i go to the school i see a bitch which get fucked by an old man "]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
x=model.predict(padded)
print(x)
if x>0.5:
    print("apprpriate context")
else:
    print("inappropriate context")


[[0.9999994]]
apprpriate context
